In [114]:
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from scipy.sparse import hstack

In [ ]:
# 0.78732 - это мой ужас внизу.
# научиться получше обрабатывать текст
# автоматизировать процесс обработки. full.fit_transform(all_data, len(data))
# автоматизировать поиск лучшей модели

In [164]:
data = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [72]:
data.shape

(7613, 5)

In [73]:
test.shape

(3263, 4)

In [142]:
all_text = list(pd.concat([data, test])['text'])

In [47]:
# all_text

In [3]:
data.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [140]:
x_2 = full_pipeline.fit_transform(pd.concat([data, test])[['keyword', 'location']])

In [75]:
data.shape

(7613, 5)

In [76]:
y = data.target

In [143]:
vectorizer = CountVectorizer()

In [144]:
x = vectorizer.fit_transform(all_text)

In [145]:
x

<10876x27922 sparse matrix of type '<class 'numpy.int64'>'
	with 159630 stored elements in Compressed Sparse Row format>

In [146]:
data.drop(['target', 'id', 'text'], axis=1, inplace=True)

In [129]:
pipe_ohe = Pipeline([
    ('ohe', OneHotEncoder())
])

In [136]:
full_pipeline = ColumnTransformer([
#     ('text', CountVectorizer(), 'text'),
    ('cat', OneHotEncoder(), code_col)
])
code_col = ['keyword', 'location']

In [132]:
data.fillna('NaN', inplace=True)

In [133]:
test.fillna('NaN', inplace=True)

In [147]:
train = hstack((x_2[:len(data), :], x[:len(data), :]))

In [ ]:
# у меня нет train/test

In [148]:
test = hstack((x_2[len(data):, :], x[len(data):, :]))

In [149]:
for_clf = RandomForestClassifier(n_estimators=100, bootstrap=True, criterion='entropy')

In [150]:
for_clf.fit(train, y)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='entropy', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [151]:
y_pred = for_clf.predict(test)

In [155]:
ids = pd.read_csv('sample_submission.csv')

In [157]:
ids['target'] = y_pred

In [160]:
ids.to_csv('answer.csv', index=False)

In [162]:
# !type answer.csv

In [166]:
data.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [ ]:
# надо удалить id, кодировать keyword и location, текст прогнать через vectorizer, target удалить

In [52]:
drop_col = ['id', 'target']
code_col = ['keyword', 'location']

In [53]:
full_pipeline = ColumnTransformer([
    ('text', CountVectorizer(), 'text'),
    ('cat', OneHotEncoder(), code_col)
])

In [54]:
# drop columns
data = data.drop(['id', 'target'], axis=1)

In [55]:
data = data.fillna('Nan')

In [56]:
pipeline = Pipeline([
    ('cols', full_pipeline)
])

In [57]:
mat = pipeline.fit_transform(data)

In [58]:
mat

<7613x25201 sparse matrix of type '<class 'numpy.float64'>'
	with 126723 stored elements in Compressed Sparse Row format>

In [59]:
lin_clf = SGDClassifier()

In [60]:
for_clf = RandomForestClassifier()

In [61]:
y2 = cross_val_score(for_clf, mat, y, cv=3)

In [62]:
y2

array([0.70449173, 0.63317573, 0.71265274])

In [83]:
params = {'n_estimators': [1, 10, 100, 1000],
         'bootstrap': [True, False],
         'criterion': ['gini', 'entropy']}

In [85]:
grid_for = GridSearchCV(for_clf, params, scoring='accuracy', cv=5, verbose=2, return_train_score=True, n_jobs=-1)

In [86]:
grid_for.fit(mat, y)

Fitting 5 folds for each of 16 candidates, totalling 80 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:  7.2min
[Parallel(n_jobs=-1)]: Done  80 out of  80 | elapsed: 41.5min finished


GridSearchCV(cv=5, error_score=nan,
             estimator=RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                              class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              max_samples=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators=100, n_jobs=None,
                                              oob_score=False,
                                              rando

In [87]:
grid_for.best_estimator_

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='entropy', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [63]:
for_clf = RandomForestClassifier(n_estimators=100, bootstrap=True, criterion='entropy')

In [92]:
for_clf.fit(mat, y)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='entropy', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [ ]:
import joblib


In [97]:
test = test.fillna('Nan')

In [98]:
ids = test.id
test = test.drop(['id'], axis=1)

In [99]:
mat_test = pipeline.fit_transform(test)

In [64]:
# y_pred = for_clf.predict(mat_test)